In [1]:
import os
import os.path as path
import gc
import re
import math
import json
import random

In [2]:
from datetime import datetime
import time

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..', 'data', 'sql_dummy')
DIR_SAVE_PATH = path.join('..', 'data', 'output')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..\data\sql_dummy
..\data\output


In [6]:
# sql_bean.csv
bean_read = pd.read_csv(path.join(DIR_PATH, 'sql_bean.csv'), low_memory=False, encoding='cp949')

print(bean_read.shape)
bean_read.head()

(47, 8)


,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade
0,1,NaN,NaN,에티오피아 예가체프 G2 워시드 (중배전),에티오피아 예가체프 G2 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
1,2,NaN,NaN,과테말라 안티구아 SHB 워시드 (강배전),과테말라 안티구아 SHB 워시드 (강배전),아이템 요약 영역 입니다!!!,default_bean.png,0
2,3,NaN,NaN,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아 코케 G1 펄프드내추럴 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
3,4,NaN,NaN,케냐 키암부 AA 워시드 (중배전),케냐 키암부 AA 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
4,5,NaN,NaN,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0


In [7]:
# sql_like_list.csv
like_list_read = pd.read_csv(path.join(DIR_PATH, 'sql_like_list.csv'), low_memory=False, encoding='cp949')

print(like_list_read.shape)
like_list_read.head()

(9366, 7)


,idx,created_date,updated_date,expired,item_idx,item_type,member_idx
0,1,NaN,NaN,N,1,bean,3
1,2,NaN,NaN,N,1,bean,6
2,3,NaN,NaN,N,1,bean,10
3,4,NaN,NaN,N,1,bean,13
4,5,NaN,NaN,N,1,bean,14


In [8]:
like_list_df = pd.merge(like_list_read, bean_read[['idx', 'name_ko']], left_on='item_idx', right_on='idx')
like_list_df.head()

,idx_x,created_date,updated_date,expired,item_idx,item_type,member_idx,idx_y,name_ko
0,1,NaN,NaN,N,1,bean,3,1,에티오피아 예가체프 G2 워시드 (중배전)
1,2,NaN,NaN,N,1,bean,6,1,에티오피아 예가체프 G2 워시드 (중배전)
2,3,NaN,NaN,N,1,bean,10,1,에티오피아 예가체프 G2 워시드 (중배전)
3,4,NaN,NaN,N,1,bean,13,1,에티오피아 예가체프 G2 워시드 (중배전)
4,5,NaN,NaN,N,1,bean,14,1,에티오피아 예가체프 G2 워시드 (중배전)


In [9]:
like_list_df = like_list_df[['item_idx', 'member_idx', 'name_ko']]
like_list_df = like_list_df.pivot(index=['member_idx'], columns=['item_idx', 'name_ko'], values='member_idx')
like_list_df.fillna(0, inplace=True)
like_list_df = like_list_df.astype(bool).astype(int)

In [10]:
#user_base로 변환
like_list_df = like_list_df.T
like_list_df.head()

,member_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000
item_idx,name_ko,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,에티오피아 예가체프 G2 워시드 (중배전),0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,과테말라 안티구아 SHB 워시드 (강배전),0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,1,...,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,에티오피아 코케 G1 펄프드내추럴 (중배전),1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,0,0
4,케냐 키암부 AA 워시드 (중배전),1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),0,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,...,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1


In [11]:
%%time
like_cosine_sim = cosine_similarity(like_list_df)

CPU times: total: 0 ns
Wall time: 1.92 ms


In [12]:
print(like_cosine_sim.shape)
print(like_cosine_sim.dtype)

like_cosine_sim = like_cosine_sim.astype(np.float16)
gc.collect()
print(like_cosine_sim.dtype)

like_cosine_sim

(47, 47)
float64
float16


array([[1.    , 0.2201, 0.201 , ..., 0.2281, 0.1877, 0.2578],
       [0.2201, 1.    , 0.1782, ..., 0.1777, 0.1809, 0.224 ],
       [0.201 , 0.1782, 1.    , ..., 0.2081, 0.1907, 0.1808],
       ...,
       [0.2281, 0.1777, 0.2081, ..., 1.    , 0.1896, 0.2129],
       [0.1877, 0.1809, 0.1907, ..., 0.1896, 1.    , 0.1733],
       [0.2578, 0.224 , 0.1808, ..., 0.2129, 0.1733, 1.    ]],
      dtype=float16)

In [13]:
# 제목 검색 뿐만 아니라 id로도 검색할 수 있도록 행과 열 중 하나를 name_ko, 다른 하나를 item_idx로 지정
# multi index의 droplevel을 사용해서 index를 분리 (title 기준, movieId 기준으로 총 2개로 분리)
like_cosine_sim_df = pd.DataFrame(like_cosine_sim, index = like_list_df.droplevel(1).index, columns = like_list_df.droplevel(0).index, dtype=np.float16)

In [14]:
print(like_cosine_sim_df.shape)
like_cosine_sim_df.head()

(47, 47)


name_ko,에티오피아 예가체프 G2 워시드 (중배전),과테말라 안티구아 SHB 워시드 (강배전),에티오피아 코케 G1 펄프드내추럴 (중배전),케냐 키암부 AA 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),페루 피우라 MICRO LOT 워시드 (중배전),브라질 세하도 NY2 FC 내추럴 (중배전),케냐 키암부 FAQ 워시드 (중배전),파나마 보케테 SPECIALTY 워시드 (중배전),브라질 세하도 NY2 FC 펄프드내추럴 (중배전),브라질 세하도 SPECIALTY 펄프드내추럴 (중배전),코스타리카 나란조 MICRO LOT 워시드 (중배전),온두라스 엘 파라이소 SPECIALTY 워시드 (중배전),에티오피아 시다모 SPECIALTY 워시드 (중배전),과테말라 우에우에테낭고 SHB 워시드 디카페인 (강배전),콜롬비아 후일라 EXCELOS 워시드 디카페인 (강배전),인도 말라바르 AA 워시드 (중배전),코스타리카 따라주 SHB 워시드 (중배전),르완다 부산제 SPECIALTY 워시드 (중배전),브라질 카르모 데 미나스 - 워시드 디카페인 (중배전),엘살바도르 산타아 MICRO LOT 워시드 (중배전),인도네시아 수마트라 G1 웻헐드 (중배전),베트남 베트남 G1 워시드 (중배전),에티오피아 시다모 G2 워시드 (중배전),인도 치크마갈루르 AA 워시드 (중배전),에티오피아 시다모 G2 워시드 디카페인 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 디카페인 (강배전),에티오피아 예가체프 G2 워시드 (약배전),브라질 세하도 NY2 FC 네추럴 (중배전),과테말라 안티구아 SHB 워시드 (중배전),콜롬비아 후일라 SUPREMO 워시드 (중배전),베트남 다낭 G1 폴리싱 (강배전),브라질 산토스 NY2 FC 내추럴 (중배전),에티오피아 시다모 G4 내추럴 (중배전),에티오피아 예가체프 G4 내추럴 (중배전),탄자니아 음빙가 AA 워시드 (강배전),콜롬비아 메데인 SUPREMO 워시드 (강배전),코스타리카 따라주 SHB 워시드 (강배전),파푸아뉴기니 와기벨리 AA 워시드 (강배전),인도네시아 만델링 G1 워시드 (강배전),온두라스 산티아고 푸링글라 SHB 워시드 (강배전),엘 살바도르 엘 살바도르 SHG 워시드 (강배전),케냐 키암부 AA 워시드 (강배전),콜롬비아 메데인 SUPREMO 워시드 디카페인 (강배전),과테말라 안티구아 SHB 워시드 디카페인 (강배전),에티오피아 예가체프 G2 워시드 디카페인 (강배전),브라질 산토스 NY2 FC 워시드 디카페인 (강배전)
item_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.220093,0.201050,0.197632,0.199097,0.193237,0.188232,0.165161,0.221924,0.174194,0.174316,0.211304,0.213623,0.240967,0.179199,0.202637,0.207275,0.203369,0.151733,0.235718,0.174194,0.177734,0.212280,0.189575,0.211792,0.209839,0.175171,0.208496,0.179688,0.197632,0.213013,0.190063,0.183228,0.176514,0.244507,0.233887,0.237671,0.216309,0.208984,0.170532,0.172363,0.200439,0.192261,0.224365,0.228149,0.187744,0.257812
2,0.220093,1.000000,0.178223,0.190918,0.212158,0.232788,0.171265,0.249390,0.230835,0.177246,0.198120,0.194946,0.153076,0.245117,0.212646,0.195557,0.205811,0.177368,0.207275,0.213745,0.167114,0.170166,0.230347,0.239868,0.220215,0.178711,0.217773,0.156616,0.177734,0.150757,0.176392,0.222290,0.206543,0.189575,0.192993,0.194214,0.216675,0.185059,0.203125,0.178589,0.170166,0.235229,0.166260,0.251953,0.177734,0.180908,0.223999
3,0.201050,0.178223,1.000000,0.195923,0.218506,0.168945,0.153931,0.177002,0.194580,0.122742,0.159302,0.152832,0.223755,0.191162,0.176147,0.150391,0.162598,0.160889,0.198120,0.203247,0.192139,0.179321,0.192139,0.223022,0.206909,0.209229,0.208618,0.202271,0.176514,0.222412,0.185791,0.203735,0.191162,0.173462,0.149780,0.163696,0.143311,0.168579,0.189087,0.161377,0.212036,0.148804,0.185547,0.170410,0.208130,0.190674,0.180786
4,0.197632,0.190918,0.195923,1.000000,0.219116,0.208984,0.223877,0.164551,0.171021,0.225098,0.179077,0.202026,0.159668,0.198120,0.184204,0.208252,0.197510,0.228882,0.214355,0.173828,0.179077,0.177246,0.218140,0.228027,0.183838,0.195557,0.210083,0.178589,0.158936,0.192871,0.193359,0.205078,0.229004,0.166260,0.184570,0.196167,0.229004,0.141357,0.205200,0.195923,0.166748,0.174316,0.162964,0.225708,0.214478,0.192871,0.211792
5,0.199097,0.212158,0.218506,0.219116,1.000000,0.210571,0.219727,0.182495,0.208862,0.196289,0.171753,0.188965,0.210571,0.209106,0.220825,0.184448,0.204468,0.181396,0.224243,0.197021,0.215942,0.195801,0.227905,0.186890,0.269043,0.211670,0.230347,0.195801,0.245972,0.175293,0.190430,0.192017,0.209961,0.178833,0.162354,0.216431,0.244141,0.198730,0.242554,0.182983,0.219971,0.187500,0.213257,0.253418,0.210571,0.180176,0.184814


In [15]:
like_cosine_sim_df.iloc[:, like_cosine_sim_df.index.get_indexer([1])].sort_values(by=like_cosine_sim_df.iloc[:, like_cosine_sim_df.index.get_indexer([1])].columns[0], ascending=False)[1:11].index

Int64Index([47, 35, 14, 37, 20, 36, 45, 44, 9, 2], dtype='int64', name='item_idx')

In [16]:
# id 기반 추천 알고리즘
def recommendations_by_id(target_id, matrix, items, k=10):
    try:
        target_idx =  matrix.index.get_indexer([target_id])
        recom_idx = matrix.iloc[:, target_idx].sort_values(by= matrix.iloc[:, target_idx].columns[0], ascending=False)[1:11].index

        # 반환한 인덱스 값은 1부터 시작하나, 실제 iloc로 접근하는 인덱스 값은 0부터 시작하므로 이를 보정해야함
        recom_idx = recom_idx-1
        recom_id = items.iloc[recom_idx, :].idx.values
        recom_title = items.iloc[recom_idx, :].name_ko.values

        target_id_list = np.full(len(range(k)), target_id)
        target_title_list = np.full(len(range(k)), items[items.idx == target_id].name_ko.values)
        
    except:
        print(recom_idx)
        print(recom_id, recom_title)
        print(target_id_list, target_title_list)
    
    d = {
        'target_id': target_id_list,
        'target_title': target_title_list,
        'recom_id'    : recom_id,
        'recom_title' : recom_title,
    }
    
    return pd.DataFrame(d)

In [17]:
recommendations_by_id(3, like_cosine_sim_df, bean_read)

,target_id,target_title,recom_id,recom_title
0,3,에티오피아 코케 G1 펄프드내추럴 (중배전),13,온두라스 엘 파라이소 SPECIALTY 워시드 (중배전)
1,3,에티오피아 코케 G1 펄프드내추럴 (중배전),24,에티오피아 시다모 G2 워시드 (중배전)
2,3,에티오피아 코케 G1 펄프드내추럴 (중배전),30,과테말라 안티구아 SHB 워시드 (중배전)
3,3,에티오피아 코케 G1 펄프드내추럴 (중배전),5,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전)
4,3,에티오피아 코케 G1 펄프드내추럴 (중배전),41,온두라스 산티아고 푸링글라 SHB 워시드 (강배전)
5,3,에티오피아 코케 G1 펄프드내추럴 (중배전),26,에티오피아 시다모 G2 워시드 디카페인 (중배전)
6,3,에티오피아 코케 G1 펄프드내추럴 (중배전),27,콜롬비아 콜롬비아 SUPREMO 워시드 디카페인 (강배전)
7,3,에티오피아 코케 G1 펄프드내추럴 (중배전),45,과테말라 안티구아 SHB 워시드 디카페인 (강배전)
8,3,에티오피아 코케 G1 펄프드내추럴 (중배전),25,인도 치크마갈루르 AA 워시드 (중배전)
9,3,에티오피아 코케 G1 펄프드내추럴 (중배전),32,베트남 다낭 G1 폴리싱 (강배전)


In [18]:
# id 기반 추천 알고리즘
def recommendation_list_by_id(target_id, matrix, items, k=10):
    try:
        target_idx =  matrix.index.get_indexer([target_id])
        recom_idx = matrix.iloc[:, target_idx].sort_values(by= matrix.iloc[:, target_idx].columns[0], ascending=False)[1:11].index

        # 반환한 인덱스 값은 1부터 시작하나, 실제 iloc로 접근하는 인덱스 값은 0부터 시작하므로 이를 보정해야함
        recom_idx = recom_idx-1
        recom_id = items.iloc[recom_idx, :].idx.values
        recom_title = items.iloc[recom_idx, :].name_ko.values
        
    except:
        print(recom_idx)
        print(recom_id, recom_title)
        print(target_id_list, target_title_list)
    
    recom_list = [dict(id = id, title = title) for id, title in zip(recom_id, recom_title)]
    
    return recom_list

In [19]:
recommendation_list_by_id(3, like_cosine_sim_df, bean_read)

[{'id': 13, 'title': '온두라스 엘 파라이소 SPECIALTY 워시드 (중배전)'},
 {'id': 24, 'title': '에티오피아 시다모 G2 워시드 (중배전)'},
 {'id': 30, 'title': '과테말라 안티구아 SHB 워시드 (중배전)'},
 {'id': 5, 'title': '콜롬비아 콜롬비아 SUPREMO 워시드 (중배전)'},
 {'id': 41, 'title': '온두라스 산티아고 푸링글라 SHB 워시드 (강배전)'},
 {'id': 26, 'title': '에티오피아 시다모 G2 워시드 디카페인 (중배전)'},
 {'id': 27, 'title': '콜롬비아 콜롬비아 SUPREMO 워시드 디카페인 (강배전)'},
 {'id': 45, 'title': '과테말라 안티구아 SHB 워시드 디카페인 (강배전)'},
 {'id': 25, 'title': '인도 치크마갈루르 AA 워시드 (중배전)'},
 {'id': 32, 'title': '베트남 다낭 G1 폴리싱 (강배전)'}]

In [20]:
# 유사도 기준으로 추천 원두의 상위 5개를 출력
bean_recom = bean_read.copy()[['idx', 'name_ko']]
bean_recom['recommendation'] = bean_recom.apply(lambda x: recommendation_list_by_id(x.idx, like_cosine_sim_df, bean_read, k=5), axis=1)
print(bean_recom.shape)
bean_recom.head()

(47, 3)


,idx,name_ko,recommendation
0,1,에티오피아 예가체프 G2 워시드 (중배전),"[{'id': 47, 'title': '브라질 산토스 NY2 FC 워시드 디카페인 ..."
1,2,과테말라 안티구아 SHB 워시드 (강배전),"[{'id': 44, 'title': '콜롬비아 메데인 SUPREMO 워시드 디카페..."
2,3,에티오피아 코케 G1 펄프드내추럴 (중배전),"[{'id': 13, 'title': '온두라스 엘 파라이소 SPECIALTY 워시..."
3,4,케냐 키암부 AA 워시드 (중배전),"[{'id': 37, 'title': '콜롬비아 메데인 SUPREMO 워시드 (강배..."
4,5,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),"[{'id': 25, 'title': '인도 치크마갈루르 AA 워시드 (중배전)'}..."


In [21]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
# bean_recom.to_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_recom.csv'), sep=',')
bean_recom.to_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_by_like_recom.csv'), sep=',', index=False)

In [22]:
# bean_cbf_by_like_recom.csv
like_recom_read = pd.read_csv(path.join(DIR_SAVE_PATH, 'bean_cbf_by_like_recom.csv'), low_memory=False)

print(like_recom_read.shape)
like_recom_read.head()

(47, 3)


,idx,name_ko,recommendation
0,1,에티오피아 예가체프 G2 워시드 (중배전),"[{'id': 47, 'title': '브라질 산토스 NY2 FC 워시드 디카페인 ..."
1,2,과테말라 안티구아 SHB 워시드 (강배전),"[{'id': 44, 'title': '콜롬비아 메데인 SUPREMO 워시드 디카페..."
2,3,에티오피아 코케 G1 펄프드내추럴 (중배전),"[{'id': 13, 'title': '온두라스 엘 파라이소 SPECIALTY 워시..."
3,4,케냐 키암부 AA 워시드 (중배전),"[{'id': 37, 'title': '콜롬비아 메데인 SUPREMO 워시드 (강배..."
4,5,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),"[{'id': 25, 'title': '인도 치크마갈루르 AA 워시드 (중배전)'}..."


In [23]:
print(like_list_read.shape)
like_list_read.head()

(9366, 7)


,idx,created_date,updated_date,expired,item_idx,item_type,member_idx
0,1,NaN,NaN,N,1,bean,3
1,2,NaN,NaN,N,1,bean,6
2,3,NaN,NaN,N,1,bean,10
3,4,NaN,NaN,N,1,bean,13
4,5,NaN,NaN,N,1,bean,14


In [24]:
user_likes = like_list_read.loc[(like_list_read['member_idx'] == 1) & (like_list_read['item_type'] == 'bean')]
user_likes = list(user_likes['item_idx'].values)
user_likes

[3, 4, 9, 25]

In [25]:
# 사용자의 좋아요를 누른 원두를 기준으로 
# 같은 원두를 좋아요 누른 다른 사용자 정보를 활용해(cf)
# 좋아요 원두를 누른 다른 사용자가 좋아하는 원두를 추천해줌
def get_recom_by_user(userIdx, data, matrix, item_type='bean'):
    try:
        user_likes = data.loc[(data['member_idx'] == userIdx) & (data['item_type'] == item_type)]
        user_likes = list(user_likes['item_idx'].values)
        
        recom_list = []
        for temp_list in matrix.set_index('idx').loc[user_likes]['recommendation']:
            recom_list.extend(json.loads(temp_list.replace('\'', '\"')))
            
        recom_list = [dict(t) for t in {tuple(d.items()) for d in recom_list}]
        recom_list = random.sample(recom_list, k=5)
        
    except:
        print(user_likes)
        print(recom_list)
        
    return recom_list

In [26]:
recom_list = get_recom_by_user(1, like_list_read, like_recom_read, item_type='bean')
recom_list

[{'id': 18, 'title': '코스타리카 따라주 SHB 워시드 (중배전)'},
 {'id': 29, 'title': '브라질 세하도 NY2 FC 네추럴 (중배전)'},
 {'id': 25, 'title': '인도 치크마갈루르 AA 워시드 (중배전)'},
 {'id': 30, 'title': '과테말라 안티구아 SHB 워시드 (중배전)'},
 {'id': 2, 'title': '과테말라 안티구아 SHB 워시드 (강배전)'}]

In [27]:
recom_list = get_recom_by_user(0, like_list_read, like_recom_read, item_type='bean')
recom_list

[]
[]


[]